<a href="https://colab.research.google.com/github/mustafae03/meeting-summary-project/blob/main/meeting_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q requests torch transformers sentencepiece accelerate openai
!pip install -U bitsandbytes
!pip install -U transformers accelerate


In [2]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch


In [3]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
audio_filename = "/content/drive/MyDrive/sandiego_c715f40bcd35aa72dde1f47ee17fecfd.mp3"




In [6]:
hf_token=userdata.get('hf')
login(hf_token)

In [7]:
open_ai_key = userdata.get('openaikey')
openai = OpenAI(api_key=open_ai_key)

In [8]:
with open(audio_filename, "rb") as audio_file:
    transcript = openai.audio.transcriptions.create(
        model=AUDIO_MODEL,
        file=audio_file,
        response_format="text"
    )

print(transcript)


["Pomp and Circumstance"] ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ We have been advised by the city attorney that the two local state of emergency declarations for the landslides and for the wildfires don't last through the holiday. So we are having a meeting for that, and a couple extra items got added on, and then there's a closed session afterwards. So item 701 is the reconsideration of the effective date of the city's construction and demolition surcharge contained in the revised fee schedule and regulations for the Miramar landfill. Is there a motion? So moved. Second. Motion by Mr. Hueso, second by Mr. Medaffer. We had extensive discussion of this, changing the time of the new fees. Mr. Barnum, did you want to be heard on this? I don't need to. Okay, he's registered to support. And Ms. Fry is with us? Yeah, and I'll make the motion. Oh, we got the motion. On item 701, this is the reconsideration of the effective date. Motion. That is a motion by Mr. Hueso, second by Mr. Lands

In [9]:
user_prompt = "Lütfen katılımcılarla birlikte bir özet, yer ve tarih, tartışma noktaları, çıkarımlar ve sahipleriyle birlikte eylem öğeleri de dahil olmak üzere tutanakları işaretleyerek yaz ve kısa olmasın ayrıca tekrarlardan kaçın hayal görme sadece transkripteki bilgiler "
system_prompt =  "Sadece AŞAĞIDAKİ TRANSCRIPT'TEKİ ifadeleri kullanarak ÖZET çıkar. Metinde olmayan hiçbir bilgi EKLEME. Tahmin etme.  İsim/yer/tarih geçmiyorsa '—' yaz. Türkçe, maddeli ve kısa yaz."

In [10]:
messages = [

          { "role": "system", "content": system_prompt },
          { "role": "user", "content": user_prompt },


]

In [11]:
from transformers import BitsAndBytesConfig
import torch

quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
model = AutoModelForCausalLM.from_pretrained(LLAMA, quantization_config=quant_config, device_map="auto")
streamer = TextStreamer(tokenizer)
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
output = model.generate(inputs=input_ids, max_new_tokens=300 , streamer = streamer, temperature=0.3)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Sadece AŞAĞIDAKİ TRANSCRIPT'TEKİ ifadeleri kullanarak ÖZET çıkar. Metinde olmayan hiçbir bilgi EKLEME. Tahmin etme.  İsim/yer/tarih geçmiyorsa '—' yaz. Türkçe, maddeli ve kısa yaz.<|eot_id|><|start_header_id|>user<|end_header_id|>

Lütfen katılımcılarla birlikte bir özet, yer ve tarih, tartışma noktaları, çıkarımlar ve sahipleriyle birlikte eylem öğeleri de dahil olmak üzere tutanakları işaretleyerek yaz ve kısa olmasın ayrıca tekrarlardan kaçın hayal görme sadece transkripteki bilgiler<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Tutanaklar:

1. **ÖZET**
- Konuşma, 2023 yılında yapılan bir çalışmada ele alınan bir konu hakkında katılımcıların görüşlerini içeriyor.
- Katılımcılar, konuyla ilgili deneyimlerini paylaşıyor ve birbirleriyle tartışıyorlar.

2. **YER**
— (Belirtilmediği için)

3. **Tarih**
— (Belirtilmediği için)

4. **DİPLOMATİK DİALOG**
- K

In [13]:
def generate_meeting_summary(audio_transcript):

    system_prompt = "temel tartışma noktalarını ,  çıkarımlar vve eylem maddelerini toplantının genel özetini çıkaran bir yardımcı asistansın"
    user_prompt = "Lütfen katılımcılarla birlikte bir özet, yer ve tarih, tartışma noktaları, çıkarımlar ve sahipleriyle birlikte eylem öğeleri de dahil olmak üzere tutanakları işaretleyerek yaz ve kısa olmasın ayrıca tekrarlardan kaçın "


    messages = [
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": user_prompt },
    ]


    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    streamer = TextStreamer(tokenizer)

    output = model.generate(inputs=input_ids, max_new_tokens=512, streamer=streamer)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


    assistant_response_start = generated_text.find("<|assistant|>")
    if assistant_response_start != -1:

        return generated_text[assistant_response_start + len("<|assistant|>"):].strip()
    else:

        return generated_text.strip()

In [15]:
import gradio as gr


with gr.Blocks(title="Proje 3 - Tutanak  Özet") as ui:
    gr.Markdown("Transcript Tutanak\nWhisper çıktısını metin kutusuna yapıştır, butona bas.")

    with gr.Row():
        girdi = gr.Textbox(label="Transcript (metin)", lines=10, placeholder="Whisper çıktısını buraya yapıştır")
        buton = gr.Button("Özetle")

    cikti = gr.Textbox(label="Çıktı (Markdown)", lines=20)


    buton.click(
        fn=lambda t: generate_meeting_summary(t),
        inputs=girdi,
        outputs=cikti
    )

ui.launch(inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3f4f01c4c9864c8d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
